## Sentimental Analysis Prediction

.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer # frequency based DTM
from sklearn.feature_extraction.text import TfidfVectorizer # tf-idf based DTM
import konlpy.tag

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

twitter = konlpy.tag.Twitter()
from ckonlpy.tag import Twitter

In [2]:
dt = pd.read_csv('C:/Users/Ahn Suhwoo/Desktop/polarization/python_text/selected_all_test_others.csv', encoding='utf-8')
title = dt.article_title
title.head()

0           [미리보는 오늘]文 대통령, 트럼프와 환영만찬
1             국군 장병들과 기념 촬영하는 문재인 대통령
2    [단독] 대통령 특별사면 솔솔…청와대 "일부 고려 중이다"
3                 김대통령 "건설투자로 경제 활성화"
4        뜨거운 청와대 국민청원, 도 넘은 축구 대표팀 질타
Name: article_title, dtype: object

.

### 1.1 불용어 제거 및 단어 추출

In [3]:
tagger = Twitter()
tagger.add_dictionary(['우병우', '국정감사', '노믹스', '사실상', '책임총리', '소득주도성장', '송금', '가능',\
                       '한나라당', '새누리당', '한국당', '국민회의', '통합민주당', '민주통합당', '새정치민주연합', '더불어민주당', '민주당',\
                       '신당', '이명박근혜', '패싱', '엠비씨', '디제이피', '정권교체', '먹튀', '회전문', '낙하산', '사대강', '소주성',\
                       '통합진보당','정의당','바른미래당','민주노동당','민주노총','한국노총',\
                       'MBC', 'DJP', '정권교체', '먹튀','회전문', '낙하산', '4대강', '권한대행','국가보안법','대남','대북','대미','대중','대일','방한',\
                       '북방한계선','386','486','586','운동권','친일','안전보장회의','친노','친문','친이','친박', '자유무역협정','FTA','G20','APEC',\
                       ], 'Noun')

In [4]:
title_parsed = []

for tit in title:
    tit_morphs = []
    filtered_tit = tit.replace(',',' ').replace("'"," ").replace('"',' ').replace('<',' ').replace('>',' ').replace('.',' ')\
                      .replace('[',' ').replace(']',' ').replace('…',' ').replace('“',' ').replace('”',' ').replace('‘',' ').replace('’',' ')\
                      .replace('＂',' ').replace('＂',' ').replace('·',' ').replace('(',' ').replace(')',' ').replace('-',' ')\
                      .replace('軍','군').replace('國','국').replace('對','대').replace('南','남').replace('黨','당')\
                      .replace('公論','공론').replace('反','반').replace('現','현').replace('前','전').replace('金産','금산')\
                      .replace('全','전').replace('大','대').replace('民','민').replace('生','생').replace('心','심').replace('訪','방').replace('韓','한')\
                      .replace('國保法','국보법').replace('國','국').replace('與圈','여권').replace('聯政','연정').replace('發','발').replace('北核','북핵')\
                      .replace('國政','국정').replace('左派','좌파').replace('右派','우파').replace('左右','좌파 우파').replace('利敵','이적')\
                      .replace('親','친').replace('日','일').replace('史','사').replace('再議','재의')\
                      .replace('사설','').replace('경향포토','').replace('시사IN 제%d+호','').replace('단독','').replace('%','퍼센트')\
                      .replace('속보','').replace('단신','').replace('서소문사진관','').replace('특보','').replace('大選','대선')\
                      .replace('미리 보는 오늘','').replace('오마이포토','').replace('포토','').replace('派兵','파병')\
                      .replace('MBC','엠비씨').replace('DJP','디제이피').replace('4대강','사대강').replace('정권 교체','정권교체')\
                      .replace('김 대통령','김대중 대통령').replace('노 대통령','노무현 대통령').replace('이 대통령','이명박 대통령')\
                      .replace('박 대통령','박근혜 대통령').replace('문 대통령','문재인 대통령')\
                      .replace('金 대통령','김대중 대통령').replace('盧 대통령','노무현 대통령').replace('李 대통령','이명박 대통령')\
                      .replace('朴 대통령','박근혜 대통령').replace('文 대통령','문재인 대통령')\
                      .replace('김 정부','김대중 정부').replace('노 정부','노무현 정부').replace('이 정부','이명박 정부')\
                      .replace('박 정부','박근혜 정부').replace('문 정부','문재인 정부')\
                      .replace('金 정부','김대중 정부').replace('盧 정부','노무현 정부').replace('李 정부','이명박 정부')\
                      .replace('朴 정부','박근혜 정부').replace('文 정부','문재인 정부')\
                      .replace('김대통령','김대중 대통령').replace('노대통령','노무현 대통령').replace('이대통령','이명박 대통령')\
                      .replace('박대통령','박근혜 대통령').replace('문대통령','문재인 대통령')\
                      .replace('金대통령','김대중 대통령').replace('盧대통령','노무현 대통령').replace('李대통령','이명박 대통령')\
                      .replace('朴대통령','박근혜 대통령').replace('文대통령','문재인 대통령')\
                      .replace('김정부','김대중 정부').replace('노정부','노무현 정부').replace('이정부','이명박 정부')\
                      .replace('박정부','박근혜 정부').replace('문정부','문재인 정부')\
                      .replace('金정부','김대중 정부').replace('盧정부','노무현 정부').replace('李정부','이명박 정부')\
                      .replace('朴정부','박근혜 정부').replace('文정부','문재인 정부')\
                      .replace('김 정권','김대중 정권').replace('노 정권','노무현 정권').replace('이 정권','이명박 정권')\
                      .replace('박 정권','박근혜 정권').replace('문 정권','문재인 정권')\
                      .replace('金 정권','김대중 정권').replace('盧 정권','노무현 정권').replace('李 정권','이명박 정권')\
                      .replace('朴 정권','박근혜 정권').replace('文 정권','문재인 정권')\
                      .replace('김정권','김대중 정권').replace('노정권','노무현 정권').replace('이정권','이명박 정권')\
                      .replace('박정권','박근혜 정권').replace('문정권','문재인 정권')\
                      .replace('金정권','김대중 정권').replace('盧정권','노무현 정권').replace('李정권','이명박 정권')\
                      .replace('朴정권','박근혜 정권').replace('文정권','문재인 정권')\
                      .replace('靑','청와대').replace('法','법').replace('稅制','세제').replace('後','후')\
                      .replace('여권','여당').replace('與','여당').replace('野','야당').replace('北','북한').replace('檢','검찰')\
                      .replace('국민의 정부','김대중 정부').replace('참여 정부','노무현 정부').replace('DJ','김대중').replace('MB','이명박')\
                      .replace('국민의정부','김대중 정부').replace('참여정부','노무현 정부')\
                      .replace('KBS','').replace('SBS','').replace('조선일보','').replace('중앙일보','')\
                      .replace('동아일보','').replace('세계일보','').replace('아시아경제','').replace('데일리안','')\
                      .replace('한겨레','').replace('경향신문','').replace('국민일보','').replace('문화일보','').replace('서울신문','')\
                      .replace('한국일보','').replace('한국경제','').replace('매일경제','').replace('서울경제','').replace('헤럴드경제','')\
                      .replace('TV조선','').replace('채널A','').replace('MBN','').replace('JTBC','').replace('노컷뉴스','')\
                      .replace('미디어오늘','').replace('오마이뉴스','').replace('프레시안','').replace('시사IN','').replace('뉴스타파','')\
                      .replace('파이낸셜뉴스','').replace('머니투데이','').replace('이데일리','')\
                      .replace('뉴스쇼 판','').replace('경향포토','')\
                      .replace('김대중','').replace('노무현','').replace('이명박','').replace('박근혜','').replace('문재인','')\
                      .replace('%d차','').replace('%d일','').replace('촛불집회','촛불 집회').replace('국감','국정감사')\
                      .replace('특검법','특검').replace('특별검사','특검').replace('특별 검사','특검')\
                      .replace('더민주','더불어민주당').replace('새정연','새정치민주연합').replace('새정치연합','새정치민주연합')\
                      .replace('자유한국당','한국당').replace('한나라','한나라당').replace('새누리','새누리당')\
                      .replace('통합민주','통합민주당').replace('민주통합','민주통합당').replace('근혜','이명박근혜')\
                      .replace('통진당','통합진보당').replace('바른미래','바른미래당').replace('민노당','민주노동당').replace('민노총','민주노총')\
                      .replace('NLL','북방한계선').replace('NSC','안전보장회의').replace('FTA','자유무역협정')
                        
    tit_pos = tagger.pos(filtered_tit)
    for word, pos in tit_pos:
        if pos == 'Noun':
            tit_morphs.append(word)
#        if pos == 'Verb':
#            tit_morphs.append(word)
#        if pos == 'Adjective':
#            tit_morphs.append(word)
    tit_morphs_re = " ".join(tit_morphs)
    title_parsed.append(tit_morphs_re)

In [5]:
title_compare = pd.DataFrame.transpose(pd.DataFrame([title_parsed, list(title)]))
title_compare.head()

,0,1
0,미리 오늘 대통령 트럼프 환영 만찬,"[미리보는 오늘]文 대통령, 트럼프와 환영만찬"
1,국군 장병 기념 촬영 대통령,국군 장병들과 기념 촬영하는 문재인 대통령
2,대통령 특별사면 솔솔 청와대 일부 고려 중이,"[단독] 대통령 특별사면 솔솔…청와대 ""일부 고려 중이다"""
3,대통령 건설 투자 경제 활성화,"김대통령 ""건설투자로 경제 활성화"""
4,청와대 국민 청원 도 축구 대표팀 질타,"뜨거운 청와대 국민청원, 도 넘은 축구 대표팀 질타"


.

### 2. DTM 만들기 (frequency-based, tf-idf)

In [10]:
#frequency based DTM

def tf_extractor(corpus):
    vectorizer = CountVectorizer(min_df=1, ngram_range=(1,1))
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features  

filtered_texts = title_parsed
filtered_labels = dt.coded_n[0:21734]

num_train = len(filtered_labels)

train_texts = filtered_texts[:num_train]
train_labels = filtered_labels[:num_train]
test_texts = filtered_texts[num_train:]

tf_vectorizer, train_tf_features = tf_extractor(train_texts)
test_tf_features = tf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [9]:
#TF-IDF base DTM

def tfidf_extractor(corpus):
    vectorizer = TfidfVectorizer(min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=(1,1))
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

filtered_texts = title_parsed
filtered_labels = dt.coded_n[0:21734]

num_train = len(filtered_labels)

train_texts = filtered_texts[:num_train]
train_labels = filtered_labels[:num_train]
test_texts = filtered_texts[num_train:]

tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tfidf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

.

### 3. Prediction: Logistic regression (ridge, lasso)

In [11]:
# Ridge regression
# lr = LogisticRegression(C=1000.0, random_state=0)

# Lasso regression
lr = LogisticRegression(C=0.1, penalty='l1', random_state=0)

lr.fit(train_tf_features, train_labels)
pred_labels = lr.predict(test_tf_features)

#lr.fit(train_tfidf_features, train_labels)
#pred_labels = lr.predict(test_tfidf_features)

In [12]:
# Get coefficients of the model
coefficients = lr.coef_.tolist()
sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

print(sorted_coefficients[:5])
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(2145, 1.6302452143307917), (11121, 1.4341902919592355), (7506, 1.4335184473616023), (5623, 1.404853753905716), (11187, 1.3519052026988672)]
논란 (1.630)
하락 (1.434)
위반 (1.434)
소환 (1.405)
한국당 (1.352)
비판 (1.258)
강행 (1.196)
한나라당 (1.159)
파문 (1.061)
의혹 (1.060)
정권 (1.054)
사찰 (1.005)
김성태 (0.993)
탄핵 (0.951)
홍준표 (0.873)
측근 (0.853)
발언 (0.826)
손학규 (0.818)
최저 (0.808)
민주 (0.735)
사저 (0.728)
곤혹 (0.713)
불참 (0.711)
언론 (0.710)
갈등 (0.700)
특검 (0.636)
고발 (0.600)
반발 (0.589)
자금 (0.563)
의원 (0.557)
해명 (0.555)
지지율 (0.554)
개입 (0.550)
철회 (0.549)
요구 (0.532)
정부 (0.516)
기자 (0.507)
탈당 (0.488)
엠비씨 (0.472)
민심 (0.460)
장악 (0.440)
민주당 (0.438)
까지 (0.429)
사과 (0.424)
낙하산 (0.411)
검찰 (0.402)
실패 (0.399)
쓴소리 (0.374)
퍼센트 (0.354)
내곡동 (0.342)
내년 (-0.095)
연설 (-0.113)
과제 (-0.115)
경제 (-0.119)
개편 (-0.131)
평양 (-0.139)
내정 (-0.148)
업무 (-0.150)
차관 (-0.154)
적극 (-0.180)
청와대 (-0.183)
이명박근혜 (-0.189)
평화 (-0.195)
시대 (-0.199)
수석 (-0.210)
남북 (-0.234)
강화 (-0.238)
정상 (-0.244)
노력 (-0.253)
트럼프 (-0.261)
위해 (-0.272)
오바마 (-0.289)
개혁 (-0.290)
출범 (-0.295)
대

In [13]:
new_labels = list(train_labels)+list(pred_labels)

In [14]:
output = pd.DataFrame.transpose(pd.DataFrame([list(dt.set), list(dt.id), list(dt.article_id), list(dt.article_date), list(dt.article_outlet),\
                                              list(dt.article_president),list(dt.article_title), title_parsed, list(dt.coded_n), new_labels]))
output.head()

,0,1,2,3,4,5,6,7,8,9
0,1,2,138678,2017-06-29,중앙일보,문재인,"[미리보는 오늘]文 대통령, 트럼프와 환영만찬",미리 오늘 대통령 트럼프 환영 만찬,0,0
1,1,6,28344,2017-09-28,노컷뉴스,문재인,국군 장병들과 기념 촬영하는 문재인 대통령,국군 장병 기념 촬영 대통령,0,0
2,1,8,6763,2013-01-08,TV조선,이명박,"[단독] 대통령 특별사면 솔솔…청와대 ""일부 고려 중이다""",대통령 특별사면 솔솔 청와대 일부 고려 중이,0,0
3,1,12,112983,2001-05-03,조선일보,김대중,"김대통령 ""건설투자로 경제 활성화""",대통령 건설 투자 경제 활성화,0,0
4,1,17,142961,2018-06-19,중앙일보,문재인,"뜨거운 청와대 국민청원, 도 넘은 축구 대표팀 질타",청와대 국민 청원 도 축구 대표팀 질타,0,0


In [15]:
output.columns = ['set','id','article_id','article_date', 'article_outlet', 'article_president', 'article_title', 'title_parsed',\
                  'original_labels', 'new_labels']

In [16]:
output.to_csv('selected_all_test_result_others.csv')

.